In [69]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd
from probreg import gmmtree
from probreg import filterreg
from probreg import l2dist_regs
from timeit import default_timer as timer

In [70]:
# # load source and target point cloud
# source = o3.io.read_point_cloud('bunny.pcd')
# target = copy.deepcopy(source)

In [75]:
# load source and target point cloud
mesh_source = o3.io.read_triangle_mesh('test2.obj')
source = mesh_source.sample_points_uniformly(2048)

mesh_target = o3.io.read_triangle_mesh('test.obj')
target = mesh_target.sample_points_uniformly(2048)

In [76]:
# transform target point cloud
th = np.deg2rad(30.0)
target.transform(np.array([[np.cos(th), -np.sin(th), 0.0, 0.0],
                           [np.sin(th), np.cos(th), 0.0, 0.0],
                           [0.0, 0.0, 1.0, 0.0],
                           [0.0, 0.0, 0.0, 1.0]]))
source = source.voxel_down_sample(voxel_size=0.005)
target = target.voxel_down_sample(voxel_size=0.005)

In [77]:
# compute cpd registration

start = timer()

tf_param, _, _ = cpd.registration_cpd(source, target)
#tf_param, _ = gmmtree.registration_gmmtree(source, target, maxiter=150)
#tf_param, _, _ = filterreg.registration_filterreg(source, target, maxiter=100)
#tf_param = l2dist_regs.registration_gmmreg(source, target, tf_type_name='rigid')
result = copy.deepcopy(source)
result.points = tf_param.transform(result.points)

end = timer()

print("Total tunning time: {:4f}".format(end - start))

Total tunning time: 6.822400


In [78]:
# draw result
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0, 1])
o3.visualization.draw_geometries([source, target, result])